In [ ]:
import json

import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.speedups
from branca.colormap import LinearColormap
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from shapely.geometry import LineString, Polygon
from shapely.ops import substring, unary_union

if shapely.speedups.available:
    shapely.speedups.enable()

In [224]:
path_pref = "data/"

In [ ]:
# Load the GeoJSON files
midday_gdf = gpd.read_file(path_pref + "182_Midday_speeds.geojson")

In [ ]:
# This function cleans the short names in the CalTrans speed input file we use for sequence matching, as there are problematic non-numeric characters
def process_route_name(value):
    if pd.isna(value):
        print("Encountered a null value in route_short_name.")
        return value

    value = str(value)  # Convert to string to handle potential float values

    if "/" in value:
        value = value.split("/")[0]

    value = value.strip()  # Remove any leading/trailing whitespace

    if value == "":
        print("Encountered an empty string in route_short_name after processing.")
        return value

    try:
        int(value)
        # return value
    except ValueError:
        print(f"Value '{value}' in route_short_name is not convertible to an integer.")
        # return value
    return value


# Apply the function to the 'route_short_name' column
midday_gdf["route_short_name"] = midday_gdf["route_short_name"].apply(
    process_route_name
)

In [227]:
# Create a unique identifier
midday_gdf["stop_route_short_dir_id"] = (
    midday_gdf["stop_id"].astype(str)
    + "_"
    + midday_gdf["route_short_name"].astype(str)
    + "_"
    + midday_gdf["direction_id"].astype(str)
)

In [228]:
# Read GeoJSON LineString CalTrans Speed Data, save CRS, select relevant columns
full = gpd.read_file(path_pref + "metro_lines.geojson")
original_crs = full.crs
full_cols = full[
    [
        "route_id",
        "direction_id",
        "stop_pair",
        "stop_pair_name",
        "segment_id",
        "time_of_day",
        "p50_mph",
        "p80_mph",
        "route_short_name",
        "geometry",
    ]
]

In [ ]:
# Removing duplicates, keep the one with the highest difference in speed
full_cols["speed_diff"] = full_cols["p80_mph"] - full_cols["p50_mph"]
full_cols = full_cols[full_cols["time_of_day"] != "Owl"]


# Group by 'stop_route_dir_id' and get the index of the max 'speed_diff'
max_speed_diff_indices = full_cols.groupby(
    ["route_short_name", "segment_id", "time_of_day"]
)["speed_diff"].idxmax()  # route short name?

# Create a new GeoDataFrame with the rows corresponding to the max indices for each duplicate group
cleaned_gdf = full_cols.loc[max_speed_diff_indices].reset_index(drop=True)
cleaned_gdf = cleaned_gdf.drop(columns=["speed_diff"])
cleaned_gdf

In [ ]:
cleaned_gdf[cleaned_gdf["stop_pair_name"] == "5th / San Pedro__5th / Main"]

In [ ]:
cleaned_gdf[
    cleaned_gdf["stop_pair_name"] == "Santa Monica / 3rd St Promenade__5th / Colorado"
]

In [ ]:
# Group by 'segment_id' and aggregate
merged_gdf = cleaned_gdf.groupby(
    ["segment_id", "route_short_name"], as_index=False
).agg(
    {
        "route_id": "first",  # Take the first value for route_id
        "direction_id": "first",  # Take the first value for direction_id
        "stop_pair": "first",  # Take the first value for stop_pair
        "stop_pair_name": "first",  # Take the first value for stop_pair_name
        #'route_short_name': 'first',  # Take the first value for route_short_name
        "geometry": "first",  # Take the first geometry
        "p80_mph": "max",  # Take the maximum p80_mph
        #'p50_mph': 'mean',  # Take the mean of p50_mph TODO: Which values to use?
        "p50_mph": lambda x: (
            x[
                cleaned_gdf.loc[x.index, "time_of_day"].isin(
                    ["AM Peak", "PM Peak", "Midday"]
                )
            ].mean()
            or x.mean()
        )
        or 0.1,  # Fallback to overall mean if filtered mean is zero
    }
)

# Display the resulting GeoDataFrame
print("Aggregated GeoDataFrame:")
merged_gdf = gpd.GeoDataFrame(
    merged_gdf, geometry="geometry", crs=original_crs
)  # Replace "geometry" with your column name
merged_gdf

In [ ]:
# Verify there are no zero values in the 50th percentile speed for aggregated segments
print(
    "Number of segments with zero values for p50 speed:",
    len(merged_gdf[merged_gdf["p50_mph"] == 0]),
)

In [ ]:
merged_gdf[merged_gdf["stop_pair_name"] == "5th / San Pedro__5th / Main"]

In [ ]:
merged_gdf[merged_gdf["segment_id"] == "8066-8028-1"]


In [ ]:
# Filter out segments for routes beginning with 8 and 9, which are non-bus routes
merged_gdf = merged_gdf[~merged_gdf["route_id"].str.startswith("8")]
merged_gdf = merged_gdf[~merged_gdf["route_id"].str.startswith("9")]
merged_gdf

In [237]:
# Apply the short_name cleaning function to the 'route_short_name' column
merged_gdf["route_short_name"] = merged_gdf["route_short_name"].apply(
    process_route_name
)
merged_gdf["route_short_name"] = merged_gdf["route_short_name"].astype(str)

In [238]:
merged_gdf["stop_id"] = merged_gdf["stop_pair"].str.split("_").str[0]

In [ ]:
# Read the Ridership file
ridership_full = pd.read_csv(path_pref + "bus_ridership.csv")
ridership_full.head()

In [240]:
# Filter out the non-weekday data
ridership_full = ridership_full[ridership_full["DAY TYPE"] == "DX"]

In [ ]:
ridership_full["DIRECTION"].value_counts()

In [242]:
# Clean, process, and map ridership data values
ridership_full["YEAR MONTH"] = ridership_full["YEAR MONTH"].astype(str)
ridership_full["LINE"] = ridership_full["LINE"].astype(str)
ridership_full["DIRECTION_bin"] = ridership_full["DIRECTION"].map(
    {
        "North": 0.0,
        "South": 1.0,
        "East": 0.0,
        "West": 1.0,
        "Clockwise": 0.0,
        "Counterclockwise": 1.0,
    }
)
ridership_full["STOP ID"] = ridership_full["STOP ID"].astype(str)

# Create a unique identifier
ridership_full["stop_route_short_dir_id"] = (
    ridership_full["STOP ID"]
    + "_"
    + ridership_full["LINE"]
    + "_"
    + ridership_full["DIRECTION_bin"].astype(str)
)


In [243]:
# Create the same identifier in merged_gdf
merged_gdf["stop_route_short_dir_id"] = (
    merged_gdf["stop_id"].astype(str)
    + "_"
    + merged_gdf["route_short_name"].astype(str)
    + "_"
    + merged_gdf["direction_id"].astype(str)
)

In [244]:
# Clean up NaN, non-numeric, and missing values in ridership data
ridership_full["Avg Ons"] = pd.to_numeric(
    ridership_full["Avg Ons"], errors="coerce"
).fillna(0)
ridership_full["Avg Offs"] = pd.to_numeric(
    ridership_full["Avg Offs"], errors="coerce"
).fillna(0)

In [ ]:
# Aggregate ridership data to handle duplicates
ridership_agg = (
    ridership_full.groupby("stop_route_short_dir_id")
    .agg({"Avg Ons": "mean", "Avg Offs": "mean"})
    .reset_index()
)
print(ridership_agg)

# Create a dictionary for efficient mapping
ridership_dict = ridership_agg.set_index("stop_route_short_dir_id")[
    ["Avg Ons", "Avg Offs"]
].to_dict(orient="index")

# Use map to add Avg Ons and Avg Offs to merged_gdf
merged_gdf["Avg Ons"] = merged_gdf["stop_route_short_dir_id"].map(
    lambda x: ridership_dict.get(x, {}).get("Avg Ons")
)
merged_gdf["Avg Offs"] = merged_gdf["stop_route_short_dir_id"].map(
    lambda x: ridership_dict.get(x, {}).get("Avg Offs")
)


# Check if the number of rows remained the same
print(f"Original merged_gdf rows: {len(merged_gdf)}")
print(f"Final merged data rows: {len(merged_gdf)}")

# Check for any null values in the new columns
print(f"Null values in merged Avg Ons: {merged_gdf['Avg Ons'].isnull().sum()}")
print(f"Null values in merged Avg Offs: {merged_gdf['Avg Offs'].isnull().sum()}")

# Check for unique values in stop_route_short_dir_id
print(
    f"Unique stop_route_short_dir_id in merged_gdf: {merged_gdf['stop_route_short_dir_id'].nunique()}"
)
print(
    f"Unique stop_route_short_dir_id in ridership_full: {ridership_full['stop_route_short_dir_id'].nunique()}"
)

In [ ]:
# Aggregate sequence data to handle duplicates
sequence_agg = (
    midday_gdf.groupby("stop_route_short_dir_id")
    .agg(
        {
            "stop_sequence": "first",
        }
    )
    .reset_index()
)

# Create a dictionary for efficient mapping
sequence_dict = sequence_agg.set_index("stop_route_short_dir_id")[
    ["stop_sequence"]
].to_dict(orient="index")

# Use map to add sequence ids to merged_gdf
merged_gdf["stop_sequence"] = merged_gdf["stop_route_short_dir_id"].map(
    lambda x: sequence_dict.get(x, {}).get("stop_sequence")
)
merged_gdf

In [ ]:
# Inspect missing stop sequence values
merged_gdf[
    ~merged_gdf["stop_sequence"].apply(
        lambda x: isinstance(x, (int, float)) and not np.isnan(x)
    )
][
    [
        "stop_sequence",
        "stop_pair_name",
        "segment_id",
        "route_short_name",
        "stop_pair",
        "Avg Ons",
        "Avg Offs",
    ]
]

In [ ]:
"""
This block aims to fill missing sequence data. 
The stop_pair_names are split and used to fill those missing values iteratively 
"""


def fill_missing_stop_sequences(merged_gdf):
    # First, parse out the stop IDs from the stop_pair column
    merged_gdf[["first_stop_id", "second_stop_id"]] = merged_gdf["stop_pair"].str.split(
        "__", expand=True
    )

    # Create a copy of the dataframe to avoid SettingWithCopyWarning
    df = merged_gdf.copy()

    # Find rows with missing stop_sequence
    missing_sequence_mask = df["stop_sequence"].isna()
    print("Initial missing sequences:", sum(missing_sequence_mask))

    # Iterate through rows with missing stop_sequence
    for idx in df[missing_sequence_mask].index:
        # Get route and direction for the current row
        route_short_name = df.loc[idx, "route_short_name"]
        direction_id = df.loc[idx, "direction_id"]
        first_stop_id = df.loc[idx, "first_stop_id"]
        second_stop_id = df.loc[idx, "second_stop_id"]

        # Strategy 1: Find rows where first_stop matches second_stop of missing row
        # and the previous row has a known sequence
        matching_rows_forward = df[
            (df["route_short_name"] == route_short_name)
            & (df["direction_id"] == direction_id)
            & (df["stop_sequence"].notna())
            & (df["second_stop_id"] == first_stop_id)
        ]

        # Strategy 2: Find rows where second_stop matches first_stop of missing row
        # and the matching row has a known sequence
        matching_rows_backward = df[
            (df["route_short_name"] == route_short_name)
            & (df["direction_id"] == direction_id)
            & (df["stop_sequence"].notna())
            & (df["first_stop_id"] == second_stop_id)
        ]

        # If forward matching rows exist, take the sequence and add 1
        if len(matching_rows_forward) > 0:
            df.loc[idx, "stop_sequence"] = (
                matching_rows_forward["stop_sequence"].iloc[0] + 1
            )

        # If backward matching rows exist, take the sequence and subtract 1
        elif len(matching_rows_backward) > 0:
            df.loc[idx, "stop_sequence"] = (
                matching_rows_backward["stop_sequence"].iloc[0] - 1
            )

    # Count remaining missing sequences
    remaining_missing = df["stop_sequence"].isna().sum()
    return df, remaining_missing


def iteratively_fill_stop_sequences(merged_gdf):
    """
    Repeatedly run fill_missing_stop_sequences until no more sequences can be filled.

    Parameters:
    -----------
    merged_gdf : pandas.DataFrame
        Input DataFrame with transit stop information

    Returns:
    --------
    pandas.DataFrame
        DataFrame with as many stop sequences filled as possible
    """
    current_df = merged_gdf.copy()
    previous_missing_count = float("inf")

    iteration = 0
    while True:
        iteration += 1
        current_df, missing_count = fill_missing_stop_sequences(current_df)

        print(f"Iteration {iteration}: {missing_count} sequences still missing")

        # If no progress is made, break the loop
        if missing_count == previous_missing_count:
            break

        previous_missing_count = missing_count

    print(
        f"Stopped after {iteration} iterations with {missing_count} missing sequences"
    )
    return current_df


final_df = iteratively_fill_stop_sequences(merged_gdf)

In [249]:
merged_gdf = final_df

In [ ]:
"""
This function generates the ridership curve across the route segments by direction in sequence order. 
The "Ons" and "Offs" are intuitively aggregated so that we have a useful curve of the daily ridership for each route-segment-direction tuple.
One caveat is that the CalTrans data splits long segments into parts, so for segments that are created from the split we do not modify the current rider count
"""


def calculate_route_ridership(data, route, direction):
    # Filter the dataframe for the specific route and direction
    route_df = data[
        (data["route_short_name"] == str(route)) & (data["direction_id"] == direction)
    ].sort_values("stop_sequence")

    print(route_df)
    # Initialize variables
    riders_on_bus = 0
    segment_ridership = {}

    print(f"\nCalculating ridership for route {route}, direction {direction}")

    for _, row in route_df.iterrows():
        print(row["stop_sequence"])
        stop_id = int(row["stop_id"] or "0")
        stop_name = row["stop_pair_name"]
        stop_route_short_dir_id = row["stop_route_short_dir_id"]

        # Check if segment_id ends with '1', meaning it is not a split segment in the CalTrans data
        ends_with_one = str(row["segment_id"]).endswith("1")

        # Assign values based on conditions
        riders_getting_on = (
            row["Avg Ons"] if pd.notna(row["Avg Ons"]) and ends_with_one else 0
        )
        riders_getting_off = (
            row["Avg Offs"] if pd.notna(row["Avg Offs"]) and ends_with_one else 0
        )

        # Update riders on bus
        riders_on_bus += riders_getting_on - riders_getting_off
        riders_on_bus = max(0, riders_on_bus)  # Ensure it doesn't go below zero

        segment_ridership[stop_route_short_dir_id] = riders_on_bus

        print(
            f"Stop {stop_id} - {stop_name}: On: {riders_getting_on:.2f}, Off: {riders_getting_off:.2f}, On Bus: {riders_on_bus:.2f}"
        )

    return segment_ridership


# Test the function with route "720" in direction 0
test_route = "720"
test_direction = 0
result = calculate_route_ridership(merged_gdf, test_route, test_direction)

In [251]:
# Runs the ridership calculation for every route for both directions
def calculate_all_routes_ridership(merged_data):
    all_segment_ridership = {}
    # Get unique combinations of route_short_name and direction_id
    route_directions = merged_data[
        ["route_short_name", "direction_id"]
    ].drop_duplicates()

    for _, row in route_directions.iterrows():
        route = row["route_short_name"]
        direction = row["direction_id"]
        print(f"Calculating ridership for route {route}, direction {direction}")
        segment_ridership = calculate_route_ridership(merged_data, route, direction)
        all_segment_ridership.update(segment_ridership)

    return all_segment_ridership

In [ ]:
# Calculate ridership for all routes
all_routes_ridership = calculate_all_routes_ridership(merged_gdf)

# Add the calculated ridership to the original DataFrame
merged_gdf["calculated_segment_ridership"] = merged_gdf["stop_route_short_dir_id"].map(
    all_routes_ridership
)

In [ ]:
# Inspect a segment shared by two bus routes, the 2 and the 4, to see the distinct ridership count
merged_gdf[merged_gdf["segment_id"] == "8066-8028-1"]

In [ ]:
"""
Visualize the ridership curves across the network with an interactive folium map
"""
# Create a map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Create a color map
vmin = merged_gdf["calculated_segment_ridership"].min()
vmax = merged_gdf["calculated_segment_ridership"].max()
color_map = LinearColormap(colors=["blue", "yellow", "red"], vmin=vmin, vmax=vmax)


# Function to style the features
def style_function(feature):
    ridership = feature["properties"]["calculated_segment_ridership"]
    if ridership is None or np.isnan(ridership):
        color = "gray"  # Use gray for null values
    else:
        color = color_map(ridership)
    return {"color": color, "weight": 2, "fillOpacity": 0.7}


# Function to create popups
def popup_function(feature):
    props = feature["properties"]
    ridership = props["calculated_segment_ridership"]
    if ridership is None or np.isnan(ridership):
        ridership_str = "No data"
    else:
        ridership_str = f"{ridership:.2f}"
    return folium.Popup(
        f"Stop Name: {props['stop_pair_name']}<br>"
        f"Route: {props['route_short_name']}<br>"
        f"Ridership: {ridership_str}"
    )


# Add the GeoDataFrame to the map
folium.GeoJson(
    merged_gdf,
    style_function=style_function,
    popup=popup_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["stop_pair_name", "route_short_name", "calculated_segment_ridership"],
        aliases=["Stop Name:", "Route:", "Ridership:"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    ),
).add_to(m)


# Add the color map to the map
color_map.add_to(m)

# Add a title to the map
title_html = """
             <h3 align="center" style="font-size:16px"><b>Los Angeles Bus Segments</b></h3>
             """
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
# m.save("los_angeles_bus_ridership_map.html")

# Display the map (if you're in a Jupyter notebook)
m

In [ ]:
# Convert GeoDataFrame to a GeoJSON-like Python dictionary
geojson_dict = merged_gdf.__geo_interface__

# Save as GeoJSON
with open("bus_segments_calculated_ridership.geojson", "w") as f:
    json.dump(geojson_dict, f)

print("Data saved successfully to bus_segments_calculated_ridership.geojson")

In [256]:
def calculate_delay_effects(gdf):
    """
    Calculate delay effects for the consolidated segments using average p50 speed across time periods.
    """
    # Ensure the GeoDataFrame has a valid CRS
    if gdf.crs is None:
        raise ValueError("GeoDataFrame must have a CRS set.")

    # Reproject to a projected CRS for accurate length calculations (e.g., EPSG:3857 for meters)
    gdf = gdf.to_crs(epsg=3857)

    # Calculate length of LineString geometries in meters
    gdf["length_meters"] = gdf.geometry.length

    # Convert length to miles (1 meter = 0.000621371 miles)
    gdf["miles_from_last"] = gdf["length_meters"] * 0.000621371

    # Calculate difference from average speed
    gdf["diff_from_avg"] = gdf["p80_mph"] - gdf["p50_mph"]

    # Calculate actual and best travel times (hours)
    gdf["actual_time"] = gdf["miles_from_last"] / gdf["p50_mph"]
    gdf["best_time"] = gdf["miles_from_last"] / np.maximum(
        gdf["p80_mph"], 1.25 * gdf["p50_mph"]
    )

    # Calculate delay in hours and time lost in minutes
    gdf["delay_hours"] = np.maximum(0, gdf["actual_time"] - gdf["best_time"])
    gdf["time_lost"] = gdf["delay_hours"] * 60

    # Calculate total ridership minutes lost
    gdf["total_ridership_minutes_lost"] = (
        gdf["time_lost"] * gdf["calculated_segment_ridership"]
    )

    # Reproject back to original CRS if necessary
    gdf = gdf.to_crs(epsg=4326)

    return gdf

In [257]:
merged_gdf = calculate_delay_effects(merged_gdf)

In [258]:
gdf_to_merge = merged_gdf[
    [
        "stop_id",
        "stop_pair_name",
        "route_id",
        "route_short_name",
        "direction_id",
        "miles_from_last",
        "geometry",
        "stop_sequence",
        "stop_route_short_dir_id",
        "calculated_segment_ridership",
        "p50_mph",
        "p80_mph",
        "diff_from_avg",
        "time_lost",
        "total_ridership_minutes_lost",
    ]
]

In [ ]:
# Modify data types before merging segments to appropriately preserve and/or aggregate features
gdf_to_merge["direction_id"] = gdf_to_merge["direction_id"].astype(str)
gdf_to_merge["stop_sequence"] = gdf_to_merge["stop_sequence"].astype(str)
gdf_to_merge["p50_mph"] = gdf_to_merge["p50_mph"].astype(str)
gdf_to_merge["p80_mph"] = gdf_to_merge["p80_mph"].astype(str)
gdf_to_merge["diff_from_avg"] = gdf_to_merge["diff_from_avg"].astype(str)
gdf_to_merge["calculated_segment_ridership_str"] = gdf_to_merge[
    "calculated_segment_ridership"
].astype(str)

In [260]:
# Adapted from https://github.com/cal-itp/data-analyses/blob/f62b150768fb1547c6b604cb53d122531104d099/_shared_utils/shared_utils/rt_utils.py#L587-L632
def try_parallel(geometry):
    try:
        return geometry.parallel_offset(30, "right")
    except Exception:
        return geometry


def extend_line(line, extension_length):
    """Extend a line by adding length to both ends."""
    if not isinstance(line, LineString):
        return line

    coords = list(line.coords)
    if len(coords) < 2:
        return line

    # Get the direction vector at the start of the line
    start_vec = np.array(coords[1]) - np.array(coords[0])
    start_vec = start_vec / np.linalg.norm(start_vec) * extension_length
    new_start = np.array(coords[0]) - start_vec

    # Get the direction vector at the end of the line
    end_vec = np.array(coords[-1]) - np.array(coords[-2])
    end_vec = end_vec / np.linalg.norm(end_vec) * extension_length
    new_end = np.array(coords[-1]) + end_vec

    # Create new coordinates list
    new_coords = [new_start] + coords + [new_end]
    return LineString(new_coords)


def arrowize_segment(
    line_geometry, buffer_distance: int = 20, extension_length: int = 10
):
    """Given a linestring segment from a GTFS shape,
    buffer and clip to show direction of progression.

    Additionally, creates an extended version of the segment before buffering.
    """
    arrow_distance = buffer_distance  # was buffer_distance * 0.75
    st_end_distance = arrow_distance + 3  # avoid floating point errors...

    try:
        segment = line_geometry.simplify(tolerance=5)
        arrow_distance = max(arrow_distance, line_geometry.length / 20)
        shift_distance = buffer_distance + 1

        # Create extended version of the original segment
        extended_segment = extend_line(segment, extension_length)

        # Create the arrowized version
        begin_segment = substring(segment, 0, st_end_distance)
        r_shift = begin_segment.parallel_offset(shift_distance, "right")
        r_pt = substring(r_shift, 0, 0)
        l_shift = begin_segment.parallel_offset(shift_distance, "left")
        l_pt = substring(l_shift, 0, 0)
        end = substring(
            begin_segment,
            begin_segment.length + arrow_distance,
            begin_segment.length + arrow_distance,
        )
        poly = shapely.geometry.Polygon(
            (r_pt, end, l_pt)
        )  # Triangle to cut bottom of arrow

        end_segment = substring(
            segment, segment.length - st_end_distance, segment.length
        )
        end = substring(end_segment, end_segment.length, end_segment.length)
        r_shift = end_segment.parallel_offset(shift_distance, "right")
        r_pt = substring(r_shift, 0, 0)
        r_pt2 = substring(r_shift, r_shift.length, r_shift.length)
        l_shift = end_segment.parallel_offset(shift_distance, "left")
        l_pt = substring(l_shift, 0, 0)
        l_pt2 = substring(l_shift, l_shift.length, l_shift.length)
        t1 = shapely.geometry.Polygon((l_pt2, end, l_pt))
        t2 = shapely.geometry.Polygon((r_pt2, end, r_pt))

        segment_clip_mask = shapely.geometry.MultiPolygon((poly, t1, t2))

        # Buffer and clip segment with arrow shape
        differences = segment.buffer(buffer_distance).difference(segment_clip_mask)

        # Pick the largest resulting geometry
        final_geometry = max(differences.geoms, key=lambda x: x.area)

        # 2. Extended version (just buffered, no cuts)
        extended = extended_segment.buffer(buffer_distance, cap_style=2)

        return final_geometry, extended

    except Exception:
        return line_geometry.simplify(tolerance=5).buffer(
            buffer_distance
        ), line_geometry.buffer(buffer_distance)


In [261]:
# In order to programmatically merge LineStrings, we buffer them into polygons and shorten them. This later allows us to merge based on overlap.
# Note: approach changed away form this method, preserving it for now
def modify_line_geometry(gdf, reduce_length=0.1, width=0.001):
    """
    Modify line geometries in a GeoDataFrame by:
    1. Shortening lines from both ends
    2. Converting lines to thin polygons with specified width.

    Parameters:
    -----------
    gdf : GeoDataFrame
        Input GeoDataFrame with LineString geometries.
    reduce_length : float, optional (default=0.1)
        Fraction of line length to remove from each end.
    width : float, optional (default=0.001)
        Width of the resulting polygon in coordinate units.

    Returns:
    --------
    GeoDataFrame with modified polygon geometries.
    """

    def modify_line(line):
        # Calculate the total line length
        total_length = line.length

        # Compute reduction distance
        reduce_dist = total_length * reduce_length

        # Ensure we only shorten if the line is long enough
        if total_length > 2 * reduce_dist:
            # Shorten the line by removing `reduce_dist` from each end
            shortened_line = substring(line, reduce_dist, total_length - reduce_dist)
        else:
            # If the line is too short, skip shortening
            shortened_line = line

        # Buffer the line to create a thin polygon
        return shortened_line.buffer(width, cap_style=2)  # cap_style=2 for square ends

    # Create a copy of the GeoDataFrame to avoid modifying the original
    modified_gdf = gdf.copy()

    # Apply the geometry modification
    modified_gdf.geometry = modified_gdf.geometry.apply(modify_line)

    return modified_gdf

In [ ]:
# Apply the adapted Cal-ITP arrowize method for clearer looking segments post-merge
print(gdf_to_merge.crs)
# Ensure the dataset has a CRS
if gdf_to_merge.crs is None:
    gdf_to_merge.set_crs(epsg=4326, inplace=True)  # Assume WGS84 if no CRS is set

# Convert to a projected CRS for proper geometric operations
projected_crs = "EPSG:32611"  # UTM Zone 11N (meters)
gdf_to_merge = gdf_to_merge.to_crs(projected_crs)

gdf_to_merge.geometry = gdf_to_merge.geometry.apply(try_parallel)

gdf_to_merge[["geometry", "extended_geometry"]] = gdf_to_merge["geometry"].apply(
    lambda geom: pd.Series(
        arrowize_segment(geom, buffer_distance=20, extension_length=10)
    )
)


In [265]:
# Separate by direction so merging is distinct to direction of flow
gdf_to_merge_dir_zero = gdf_to_merge[gdf_to_merge["direction_id"] == "0.0"]
gdf_to_merge_dir_one = gdf_to_merge[gdf_to_merge["direction_id"] == "1.0"]

In [ ]:
def convert_crs_with_extended_geom(gdf, target_crs):
    """
    Safely converts both primary and extended geometries to the target CRS.

    Parameters:
    gdf (GeoDataFrame): Input GeoDataFrame with 'geometry' and 'extended_geometry' columns
    target_crs: Target coordinate reference system (can be EPSG code or proj string)

    Returns:
    GeoDataFrame: New GeoDataFrame with both geometries converted to target CRS
    """
    # Create a copy to avoid modifying the original
    result = gdf.copy()

    # Store the extended geometry
    extended = result["extended_geometry"]

    # First convert the main GeoDataFrame (this converts the primary geometry)
    result = result.to_crs(target_crs)

    # Create a temporary GeoDataFrame with extended_geometry as the primary geometry
    temp_gdf = gpd.GeoDataFrame(geometry=extended, crs=gdf.crs)

    # Convert the extended geometries
    converted_extended = temp_gdf.to_crs(target_crs).geometry

    # Put the converted extended geometry back
    result["extended_geometry"] = converted_extended

    return result


# Convert to geographic CRS (EPSG:4326) for visualization
gdf_to_merge_dir_zero = convert_crs_with_extended_geom(
    gdf_to_merge_dir_zero, target_crs=4326
)
gdf_to_merge_dir_one = convert_crs_with_extended_geom(
    gdf_to_merge_dir_one, target_crs=4326
)

# Check lengths in decimal degrees
print("Lengths in decimal degrees (Direction 0):")
print(gdf_to_merge_dir_zero.geometry.length.describe())
print("\nExtended geometry lengths in decimal degrees (Direction 0):")
print(gdf_to_merge_dir_zero.extended_geometry.length.describe())

print("\nLengths in decimal degrees (Direction 1):")
print(gdf_to_merge_dir_one.geometry.length.describe())
print("\nExtended geometry lengths in decimal degrees (Direction 1):")
print(gdf_to_merge_dir_one.extended_geometry.length.describe())

# Convert back to UTM for accurate length measurements
gdf_to_merge_dir_zero_utm = convert_crs_with_extended_geom(
    gdf_to_merge_dir_zero, projected_crs
)
gdf_to_merge_dir_one_utm = convert_crs_with_extended_geom(
    gdf_to_merge_dir_one, projected_crs
)

print("\nLengths in meters (Direction 0):")
print(gdf_to_merge_dir_zero_utm.geometry.length.describe())
print("\nExtended geometry lengths in meters (Direction 0):")
print(gdf_to_merge_dir_zero_utm.extended_geometry.length.describe())

print("\nLengths in meters (Direction 1):")
print(gdf_to_merge_dir_one_utm.geometry.length.describe())
print("\nExtended geometry lengths in meters (Direction 1):")
print(gdf_to_merge_dir_one_utm.extended_geometry.length.describe())

In [267]:
def merge_geom_ov_fast(gdf, overlap_threshold=0.8, debug=True):
    """
    Optimized version of merge_geom_ov using spatial indexing and sparse matrices.

    Parameters:
    gdf (GeoDataFrame): Input GeoDataFrame with polygon geometries
    overlap_threshold (float): Minimum overlap ratio required for merging (0.0 to 1.0)
    debug (bool): Whether to print debug messages

    Returns:
    GeoDataFrame: Processed GeoDataFrame with selectively merged segments
    """

    def log_debug(message):
        if debug:
            print(message)

    def create_safe_extended_geometry(regular_geom, original_ext_geoms):
        """
        Creates a valid extended geometry that properly contains the regular geometry.
        Uses original extended geometries as reference for the buffer distance.
        """
        try:
            # Calculate typical buffer distance from original geometries
            buffer_distances = []
            for reg, ext in zip(group_data.geometry, group_data.extended_geometry):
                if reg is not None and ext is not None:
                    # Find minimum distance that would make regular geometry fit in extended
                    min_dist = max(0, reg.distance(ext.boundary))
                    buffer_distances.append(min_dist)

            if buffer_distances:
                # Use median buffer distance to avoid outliers
                typical_buffer = np.median(buffer_distances)
                # Add a small safety margin
                buffer_distance = typical_buffer * 1.1
            else:
                # Fallback: use a proportion of the regular geometry's size
                buffer_distance = max(
                    regular_geom.length * 0.1, regular_geom.area**0.5 * 0.05
                )

            extended_geom = regular_geom.buffer(buffer_distance)

            # Verify the result
            if not extended_geom.contains(regular_geom):
                log_debug(
                    "Warning: Created extended geometry doesn't contain regular geometry. Increasing buffer."
                )
                extended_geom = regular_geom.buffer(buffer_distance * 1.5)

            return extended_geom

        except Exception as e:
            log_debug(f"Error creating safe extended geometry: {str(e)}")
            # Ultimate fallback: simple buffer
            return regular_geom.buffer(regular_geom.area**0.5 * 0.1)

    try:
        log_debug("Starting optimized segment merging...")

        log_debug("Type: " + str(type(gdf)))

        # Instead of checking is_valid on the Series, we'll check individual geometries
        gdf = gdf.copy()  # Make a copy to avoid modifying the original

        # Fix any invalid geometries
        for idx, row in gdf.iterrows():
            try:
                if not row.geometry.is_valid:
                    gdf.at[idx, "geometry"] = row.geometry.buffer(0)
                if not row.extended_geometry.is_valid:
                    gdf.at[idx, "extended_geometry"] = row.extended_geometry.buffer(0)
            except Exception as e:
                log_debug(f"Warning: Error fixing geometry at index {idx}: {str(e)}")

        # Create spatial index
        spatial_index = gdf.sindex

        # Pre-compute areas for all polygons
        areas = np.array([geom.area for geom in gdf.geometry])

        # Initialize sparse matrix for connectivity
        n = len(gdf)
        rows = []
        cols = []
        data = []

        # Function to check overlap ratio
        def check_overlap(idx1, idx2):
            if idx1 == idx2:
                return False

            geom1 = gdf.geometry.iloc[idx1]
            geom2 = gdf.geometry.iloc[idx2]

            if not (isinstance(geom1, Polygon) and isinstance(geom2, Polygon)):
                return False

            intersection_area = geom1.intersection(geom2).area
            min_area = min(areas[idx1], areas[idx2])

            if min_area == 0:
                return False

            return (intersection_area / min_area) >= overlap_threshold

        # Use spatial index to find potential overlaps
        for idx in range(n):
            geom = gdf.geometry.iloc[idx]
            bounds = geom.bounds

            # Get potential neighbors using spatial index
            potential_matches_idx = list(spatial_index.intersection(bounds))

            # Filter actual overlaps
            for match_idx in potential_matches_idx:
                if match_idx <= idx:  # Only check each pair once
                    continue

                if check_overlap(idx, match_idx):
                    rows.extend([idx, match_idx])
                    cols.extend([match_idx, idx])
                    data.extend([1, 1])

        # Create sparse matrix
        adjacency_matrix = csr_matrix((data, (rows, cols)), shape=(n, n))

        # Find connected components
        n_components, labels = connected_components(adjacency_matrix, directed=False)

        log_debug(f"Found {n_components} distinct groups")

        # Create new GeoDataFrame with merged geometries
        new_rows = []

        for component_id in range(n_components):
            # Get indices for this component
            component_indices = np.where(labels == component_id)[0]

            if len(component_indices) == 1:
                # Single segment, no merging needed
                new_rows.append(gdf.iloc[component_indices[0]])
            else:
                # Get the segments to merge
                group_data = gdf.iloc[component_indices]

                # Merge regular geometries
                merged_geom = unary_union(group_data.geometry)
                if not merged_geom.is_valid:
                    merged_geom = merged_geom.buffer(0)

                # Try merging extended geometries
                try:
                    merged_geom_ext = unary_union(group_data.extended_geometry)
                    if not merged_geom_ext.is_valid:
                        merged_geom_ext = merged_geom_ext.buffer(0)

                    # Check if merged extended geometry is valid and contains regular geometry
                    if merged_geom_ext.is_empty or not merged_geom_ext.contains(
                        merged_geom
                    ):
                        if merged_geom_ext.is_empty:
                            print("EMPTY")
                        log_debug(
                            f"Component {component_id}: Extended geometry invalid or doesn't contain regular geometry"
                        )
                        log_debug(
                            f"Extended geometry area: {merged_geom_ext.area}, Regular geometry area: {merged_geom.area}"
                        )
                        merged_geom_ext = create_safe_extended_geometry(
                            merged_geom, group_data.extended_geometry
                        )

                except Exception as e:
                    log_debug(
                        f"Extended geometry merge failed for component {component_id}: {str(e)}"
                    )
                    merged_geom_ext = create_safe_extended_geometry(
                        merged_geom, group_data.extended_geometry
                    )

                # Aggregate non-geometric data
                agg_data = {}
                for col in group_data.columns:
                    if col == "geometry":
                        continue
                    elif col == "miles_from_last":
                        agg_data[col] = group_data[col].max()
                    elif pd.api.types.is_numeric_dtype(group_data[col]):
                        agg_data[col] = group_data[col].sum()
                    else:
                        values = group_data[col].dropna().unique()
                        agg_data[col] = ", ".join(str(v) for v in values)

                agg_data["geometry"] = merged_geom
                agg_data["extended_geometry"] = merged_geom_ext
                new_rows.append(pd.Series(agg_data))

        # Create final GeoDataFrame
        merged_gdf = gpd.GeoDataFrame(new_rows, crs=gdf.crs)
        log_debug(f"Final segment count: {len(merged_gdf)}")

        return merged_gdf

    except Exception as e:
        log_debug(f"Error during merging: {str(e)}")
        return None

In [ ]:
"""
Visualize the ridership curves across the network with an interactive folium map
"""
# Create a map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Create a color map
vmin = gdf_to_merge_dir_zero["total_ridership_minutes_lost"].min()
vmax = gdf_to_merge_dir_zero["total_ridership_minutes_lost"].max()
color_map = LinearColormap(colors=["blue", "yellow", "red"], vmin=vmin, vmax=vmax)


# Function to style the features
def style_function(feature):
    ridership = feature["properties"]["total_ridership_minutes_lost"]
    if ridership is None or np.isnan(ridership):
        color = "gray"  # Use gray for null values
    else:
        color = color_map(ridership)
    return {"color": color, "weight": 2, "fillOpacity": 0.7}


# Function to create popups
def popup_function(feature):
    props = feature["properties"]
    ridership = props["calculated_segment_ridership"]
    if ridership is None or np.isnan(ridership):
        ridership_str = "No data"
    else:
        ridership_str = f"{ridership:.2f}"
    return folium.Popup(
        f"Stop Name: {props['stop_pair_name']}<br>"
        f"Route: {props['route_short_name']}<br>"
        f"Ridership: {ridership_str}"
    )


# Add the GeoDataFrame to the map
folium.GeoJson(
    gdf_to_merge_dir_zero,
    style_function=style_function,
    popup=popup_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["stop_pair_name", "route_short_name", "calculated_segment_ridership"],
        aliases=["Stop Name:", "Route:", "Ridership:"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    ),
).add_to(m)


# Add the color map to the map
color_map.add_to(m)

# Add a title to the map
title_html = """
             <h3 align="center" style="font-size:16px"><b>Los Angeles Bus Segments</b></h3>
             """
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
# m.save("los_angeles_bus_ridership_map.html")

# Display the map (if you're in a Jupyter notebook)
m

In [ ]:
# Runs the merge code which included the currently out-of-date extended geometry.
try:
    log_debug = lambda msg: print(f"DEBUG: {msg}")

    log_debug(f"Processing first dataset (size: {len(gdf_to_merge_dir_zero)})")
    processed = merge_geom_ov_fast(gdf_to_merge_dir_zero)

    log_debug(f"Processing second dataset (size: {len(gdf_to_merge_dir_one)})")
    processed_one = merge_geom_ov_fast(gdf_to_merge_dir_one)

    if processed is None or processed_one is None:
        print("Processing failed - check the error messages above")
    else:
        print(
            f"Successfully processed both datasets: {len(processed)} and {len(processed_one)} rows"
        )

except Exception as e:
    print(f"Unexpected error: {str(e)}")
    import traceback

    traceback.print_exc()

In [270]:
# Calculate the aggregated minutes lost normalized by the segment length
processed["minutes_lost_per_mile"] = (
    processed["total_ridership_minutes_lost"] / processed["miles_from_last"]
)
processed_one["minutes_lost_per_mile"] = (
    processed_one["total_ridership_minutes_lost"] / processed_one["miles_from_last"]
)

In [271]:
# Keeping the generation for now, but not using it in the current corridor process
processed = processed.drop(columns=["extended_geometry"])
processed_one = processed_one.drop(columns=["extended_geometry"])


In [ ]:
# Interactive network map colored by total ridership-minutes lost

# Create a map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Modify color map creation
color_map = LinearColormap(
    colors=["blue", "yellow", "red"],
    vmin=min(vmin, vmax),  # Ensure correct order
    vmax=max(vmin, vmax),
)


# Function to style the features
def style_function(feature):
    ridership = feature["properties"]["total_ridership_minutes_lost"]
    if ridership is None or np.isnan(ridership):
        color = "gray"  # Use gray for null values
    else:
        color = color_map(ridership)
    return {"color": color, "weight": 2, "fillOpacity": 0.7}


# Function to create popups
def popup_function(feature):
    props = feature["properties"]
    ridership = props["total_ridership_minutes_lost"]
    if ridership is None or np.isnan(ridership):
        ridership_str = "No data"
    else:
        ridership_str = f"{ridership:.2f}"
    return folium.Popup(
        f"Stop Name: {props['stop_name']}<br>"
        f"Route: {props['route_id']}<br>"
        f"Minutes Lost: {ridership_str}"
    )


# Define fields and aliases for tooltip (excluding 'geometry')
tooltip_fields = [
    "stop_id",
    "stop_pair_name",
    "route_id",
    "direction_id",
    "miles_from_last",
    "calculated_segment_ridership_str",
    "calculated_segment_ridership",
    "p50_mph",
    "diff_from_avg",
    "time_lost",
    "total_ridership_minutes_lost",
]

tooltip_aliases = [
    "Stop ID:",
    "Stop Name:",
    "Route ID:",
    "Direction:",
    "Miles From Last:",
    "Segment Ridership:",
    "Segment Ridership Aggregated:",
    "Average Speed (mph):",
    "Speed Difference:",
    "Time Lost (min):",
    "Total Minutes Lost:",
]

# Add the first GeoDataFrame to the map
folium.GeoJson(
    processed,
    style_function=style_function,
    popup=popup_function,
    tooltip=folium.GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=tooltip_aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
            padding: 10px;
            font-size: 12px;
        """,
        max_width=800,
    ),
).add_to(m)

# Add the second GeoDataFrame to the map
# folium.GeoJson(
#     processed_one,
#     style_function=style_function,
#     popup=popup_function,
#     tooltip=folium.GeoJsonTooltip(
#         fields=tooltip_fields,
#         aliases=tooltip_aliases,
#         localize=True,
#         sticky=False,
#         labels=True,
#         style="""
#             background-color: #F0EFEF;
#             border: 2px solid black;
#             border-radius: 3px;
#             box-shadow: 3px;
#             padding: 10px;
#             font-size: 12px;
#         """,
#         max_width=800,
#     ),
# ).add_to(m)

# Add the color map to the map
color_map.add_to(m)

# Add a title to the map
title_html = """
    <h3 align="center" style="font-size:16px"><b>Los Angeles Bus Segments</b></h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
# m.save('los_angeles_bus_map.html')

# Display the map (if you're in a Jupyter notebook)
m

In [ ]:
# Interactive network map colored by total ridership-minutes lost per mile

# Create a map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Create a color map
vmin = processed["minutes_lost_per_mile"].min()
vmax = processed["minutes_lost_per_mile"].max()
color_map = LinearColormap(colors=["blue", "yellow", "red"], vmin=vmin, vmax=vmax)


# Function to style the features
def style_function(feature):
    ridership = feature["properties"]["minutes_lost_per_mile"]
    if ridership is None or np.isnan(ridership):
        color = "gray"  # Use gray for null values
    else:
        color = color_map(ridership)
    return {"color": color, "weight": 2, "fillOpacity": 0.7}


# Function to create popups
def popup_function(feature):
    props = feature["properties"]
    ridership = props["minutes_lost_per_mile"]
    if ridership is None or np.isnan(ridership):
        ridership_str = "No data"
    else:
        ridership_str = f"{ridership:.2f}"
    return folium.Popup(
        f"Stop Name: {props['stop_name']}<br>"
        f"Route: {props['route_id']}<br>"
        f"Minutes Lost: {ridership_str}"
    )


# Define fields and aliases for tooltip (excluding 'geometry')
tooltip_fields = [
    "stop_id",
    "stop_pair_name",
    "route_id",
    "direction_id",
    "miles_from_last",
    "calculated_segment_ridership_str",
    "calculated_segment_ridership",
    "p50_mph",
    "diff_from_avg",
    "time_lost",
    "total_ridership_minutes_lost",
]

tooltip_aliases = [
    "Stop ID:",
    "Stop Name:",
    "Route ID:",
    "Direction:",
    "Miles From Last:",
    "Segment Ridership:",
    "Segment Ridership Aggregated:",
    "Average Speed (mph):",
    "Speed Difference:",
    "Time Lost (min):",
    "Total Minutes Lost:",
]

# Add the first GeoDataFrame to the map
folium.GeoJson(
    processed,
    style_function=style_function,
    popup=popup_function,
    tooltip=folium.GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=tooltip_aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
            padding: 10px;
            font-size: 12px;
        """,
        max_width=800,
    ),
).add_to(m)

# Add the second GeoDataFrame to the map
folium.GeoJson(
    processed_one,
    style_function=style_function,
    popup=popup_function,
    tooltip=folium.GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=tooltip_aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
            padding: 10px;
            font-size: 12px;
        """,
        max_width=800,
    ),
).add_to(m)

# Add the color map to the map
color_map.add_to(m)

# Add a title to the map
title_html = """
    <h3 align="center" style="font-size:16px"><b>Los Angeles Bus Segments</b></h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# Save the map
# m.save('los_angeles_bus_map.html')

# Display the map (if you're in a Jupyter notebook)
m

In [ ]:
# Top 10 segments by rider-delay for the direction '1'
processed_one.sort_values("total_ridership_minutes_lost", ascending=False).head(10)

In [ ]:
# Calculate total ridership minutes lost across all segments
total_minutes = (
    processed["total_ridership_minutes_lost"].sum()
    + processed_one["total_ridership_minutes_lost"].sum()
)

# Convert to hours for better readability
total_hours = total_minutes / 60

per_pass = total_minutes / 770000

print(f"Total passenger minutes lost: {total_minutes:,.2f}")
print(f"Total passenger hours lost: {total_hours:,.2f}")
print(f"Total minutes per passenger lost: {per_pass:,.2f}")

In [274]:
"""
This is our overall corridor ID process. Aim is to group together consecutive segments with high levels of delay per mile into clear corridor groups.
We do this by first filtering for a certain percentile threshold to get the high impact segments, then iterating across those segments in groups of streets so we only group segments running along the same street.
For each street we do a breadth first search to add segments until there is a break. Then we evaluate that grouping for whether it is significant. For us, this criteria is just whether total length is at least 1 mile.    
"""


def extract_stop_info(stop_pair_name):
    """
    Extract detailed stop information from stop_pair_name.
    Returns a list of tuples (primary_street, from_stop, to_stop) for each pair.
    """
    stop_pair_name = str(stop_pair_name)
    pairs = stop_pair_name.split(",")
    stop_info = []

    for pair in pairs:
        stops = pair.strip().split("__")
        if len(stops) == 2:
            from_stop = stops[0].strip()
            to_stop = stops[1].strip()
            # Extract primary street (before slash) from first stop
            primary_street = from_stop.split("/")[0].strip()
            stop_info.append((primary_street, from_stop, to_stop))

    return stop_info


def extract_primary_streets(stop_pair_name):
    """
    Extract unique primary street names from stop_pair_name.
    """
    stop_info = extract_stop_info(stop_pair_name)
    return list(set(info[0] for info in stop_info))


def are_segments_continuous(segment1_name, segment2_name):
    """
    Check if two segments are continuous by comparing their stop names.
    Returns True if any stop is shared between the segments.
    """
    seg1_info = extract_stop_info(segment1_name)
    seg2_info = extract_stop_info(segment2_name)

    # Get all stops from both segments
    seg1_stops = set()
    seg2_stops = set()

    for _, from_stop, to_stop in seg1_info:
        seg1_stops.add(from_stop)
        seg1_stops.add(to_stop)

    for _, from_stop, to_stop in seg2_info:
        seg2_stops.add(from_stop)
        seg2_stops.add(to_stop)

    # Check if segments share any stops
    return bool(seg1_stops.intersection(seg2_stops))


def find_continuous_adjacent_segments(segment_name, candidates_df):
    """
    Find adjacent segments that are continuously connected based on shared stops.
    """
    adjacent_segments = []

    for idx, row in candidates_df.iterrows():
        if are_segments_continuous(segment_name, row["stop_pair_name"]):
            adjacent_segments.append(idx)

    return candidates_df.loc[adjacent_segments]


def process_corridors_by_street_continuous(
    processed_df, threshold_value, threshold_percentile=75
):
    """
    Process corridors street by street ensuring continuous segments.
    """
    # Clean up any existing index columns from previous runs
    df = processed_df.copy()
    cols_to_drop = [col for col in df.columns if col.startswith("index_")]
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
    if "index" in df.columns:
        df = df.drop(columns=["index"])

    column_name = f"corridor_group_id_{threshold_percentile}"

    # Rest of the code remains the same...
    df["primary_streets"] = df["stop_pair_name"].apply(extract_primary_streets)
    all_streets = set()
    for streets in df["primary_streets"]:
        all_streets.update(streets)
    threshold = threshold_value  # df["minutes_lost_per_mile"].quantile(threshold_percentile / 100)

    corridor_groups = []
    global_group_id = 1

    for street in sorted(all_streets):
        street_mask = df["primary_streets"].apply(lambda x: street in x)
        street_df = df[street_mask].copy()
        high_impact_df = street_df[
            street_df["minutes_lost_per_mile"] >= threshold
        ].copy()

        if high_impact_df.empty:
            continue

        visited = set()
        for idx, row in high_impact_df.iterrows():
            if idx in visited:
                continue

            current_group = [idx]
            current_length = row["miles_from_last"]
            queue = [idx]
            visited.add(idx)

            while queue:
                current_segment_idx = queue.pop()
                current_segment = high_impact_df.loc[current_segment_idx]
                neighbors = find_continuous_adjacent_segments(
                    current_segment["stop_pair_name"],
                    high_impact_df[~high_impact_df.index.isin(visited)],
                )

                for neighbor_idx, neighbor in neighbors.iterrows():
                    if neighbor_idx not in visited:
                        queue.append(neighbor_idx)
                        current_group.append(neighbor_idx)
                        current_length += neighbor["miles_from_last"]
                        visited.add(neighbor_idx)

            if current_length >= 1:
                corridor_groups.extend(
                    [(idx, global_group_id, street) for idx in current_group]
                )
                global_group_id += 1

    # Create results DataFrame and merge carefully
    group_df = pd.DataFrame(
        corridor_groups, columns=["index", column_name, "primary_street"]
    )
    result = df.reset_index().merge(group_df, on="index", how="left")
    result[column_name] = result[column_name].fillna(0).astype(int)
    result = result.set_index("index")

    return result


In [ ]:
# Next few cells run the corridor ID code for different threshold values and merge together.
combined_for_threshold = pd.concat([processed, processed_one])
threshold_value_85 = combined_for_threshold["minutes_lost_per_mile"].quantile(0.85)
threshold_value_90 = combined_for_threshold["minutes_lost_per_mile"].quantile(0.90)
threshold_value_95 = combined_for_threshold["minutes_lost_per_mile"].quantile(0.95)
print(threshold_value_85, threshold_value_90, threshold_value_95)

In [276]:
street_approach_90 = process_corridors_by_street_continuous(
    processed, threshold_value_90, 90
)
street_approach_one_90 = process_corridors_by_street_continuous(
    processed_one, threshold_value_90, 90
)


In [277]:
street_approach_85 = process_corridors_by_street_continuous(
    processed, threshold_value_85, 85
)
street_approach_one_85 = process_corridors_by_street_continuous(
    processed_one, threshold_value_85, 85
)

In [278]:
street_approach_95 = process_corridors_by_street_continuous(
    processed, threshold_value_95, 95
)
street_approach_one_95 = process_corridors_by_street_continuous(
    processed_one, threshold_value_95, 95
)

In [283]:
# Reset index for all dataframes to avoid duplicate index issues
dir1_base = processed.reset_index()
dir1_85 = street_approach_85.reset_index().rename(columns={"index": "original_index"})
dir1_90 = street_approach_90.reset_index().rename(columns={"index": "original_index"})
dir1_95 = street_approach_95.reset_index().rename(columns={"index": "original_index"})

dir2_base = processed_one.reset_index()
dir2_85 = street_approach_one_85.reset_index().rename(
    columns={"index": "original_index"}
)
dir2_90 = street_approach_one_90.reset_index().rename(
    columns={"index": "original_index"}
)
dir2_95 = street_approach_one_95.reset_index().rename(
    columns={"index": "original_index"}
)

# Merge the first direction datasets using the explicit merge function
dir1_merged = dir1_base.merge(
    dir1_85[["original_index", "corridor_group_id_85"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

dir1_merged = dir1_merged.merge(
    dir1_90[["original_index", "corridor_group_id_90"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

dir1_merged = dir1_merged.merge(
    dir1_95[["original_index", "corridor_group_id_95"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

# Merge the second direction datasets
dir2_merged = dir2_base.merge(
    dir2_85[["original_index", "corridor_group_id_85"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

dir2_merged = dir2_merged.merge(
    dir2_90[["original_index", "corridor_group_id_90"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

dir2_merged = dir2_merged.merge(
    dir2_95[["original_index", "corridor_group_id_95"]],
    left_on="index",
    right_on="original_index",
    how="left",
).drop("original_index", axis=1)

# Combine both directions
combined_results = pd.concat([dir1_merged, dir2_merged], axis=0)

In [ ]:
import folium
import geopandas as gpd
from branca.colormap import linear

# Load your GeoDataFrame (assuming it's already loaded as gdf)
gdf = combined_results[
    combined_results["corridor_group_id_90"] != 0
]  # Replace with your actual GeoDataFrame variable


# Compute the center of the map
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]

# Create a folium map
m = folium.Map(location=map_center, zoom_start=11, tiles="cartodbpositron")

# Define a color scale based on corridor_group_id
corridor_ids = gdf["corridor_group_id_90"].unique()
color_scale = linear.Set1_09.scale(min(corridor_ids), max(corridor_ids))

# Add segments to the map
for _, row in gdf.iterrows():
    color = color_scale(row["corridor_group_id_90"])
    folium.GeoJson(
        row["geometry"],
        style_function=lambda feature, color=color: {
            "color": color,
            "weight": 2,
            "fillOpacity": 0.6,
        },
        tooltip=f"Corridor Group: {row['corridor_group_id_90']}<br>Name: {row['stop_pair_name']}<br>Delay: {row['total_ridership_minutes_lost']}",
    ).add_to(m)

# Add the color legend
color_scale.caption = "Corridor Group ID"
color_scale.add_to(m)

# Save map to file or display
m.save("la_metro_corridors.html")
m


In [285]:
# Convert both to WGS 84
# processed = processed.to_crs("EPSG:4326")
# processed_one = processed_one.to_crs("EPSG:4326")

# # Now concatenate
# recombined = pd.concat([processed, processed_one], ignore_index=True)

# Export to GeoJSON
combined_results.to_file("combined_routes.geojson", driver="GeoJSON")